### ECE/CS/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2022 ###

# Fantasy Football Team Drafting Opt #

#### Patrick Schmidt (pwschmidt@wisc.edu), Margaret Shen (mshen42@wisc.edu), Xingpeng Sun(xsun273@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Fantasy football is a widely played game in America where a set of league “owners” draft actual National Football League players to their fantasy teams and accrue points each week based on the on-field performance of those players. A common format of drafting fantasy teams is the snake draft. This means each owner is randomly given a pick number which is the order they draft in. This order flips for every round of the draft (so the owner that picked last in the previous round picks first in the next round). Our group is trying to use an optimization model to draft the highest scoring fantasy football team possible based on the number of points each player is projected to score at the beginning of the year. This model will be designed to determine the highest value player available for each round of a snake fantasy draft based on the currently available players and the players the team has already drafted.

![fixit flowchart][flow]

[flow]: https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/LLVPM5QCO5GFTB645NOBPMHXPM.jpg&w=1440

## 2. Mathematical model ##

We are trying to decide which players to draft in order to obtain a team with the highest value (objective). The decision variables of our problem will be a binary decision variable indicating whether or not to draft a certain player on our team. For our constraints we have: 1 kicker, 2 quarterbacks, 5 running backs, 5 wide receivers, 2 tight ends, and 1 defense.


LP model:

Parameters and variables:

$K:$ the points for all kickers;

$n_k$: the total number of kickers;

$X_k:$ the choice of each kickers, 0 if the player is not selected, 1 if the player is selected;

$Q:$ the points for all quarterbacks;

$n_k$: the total number of quarterbacks;

$X_q:$ the choice of each quarterbacks, 0 if the player is not selected, 1 if the player is selected;

$R:$ the points for all running backs;

$n_r$: the total number of running backs;

$X_r:$ the choice of each running backs, 0 if the player is not selected, 1 if the player is selected;

$W:$ the points for all wide receivers;

$n_w$: the total number of wide receivers;

$X_w:$ the choice of each wide receivers, 0 if the player is not selected, 1 if the player is selected;

$T:$ the points for all tight ends;

$n_t$: the total number of tight ends;

$X_t:$ the choice of each tight ends, 0 if the player is not selected, 1 if the player is selected;

$D:$ the points for all defense;

$n_d$: the total number of defense;

$X_d:$ the choice of each defense, 0 if the player is not selected, 1 if the player is selected;



$$
\begin{aligned}
\underset{X_k,X_q,X_r,X_w,X_t,X_d \in \mathbb{R^n}}{\text{max}}\qquad& \sum_{i=1}^{i=n_k} K(i)*X_k(i)+ \sum_{i=1}^{i=n_q}Q(i)*X_q(i)+\sum_{i=1}^{i=n_r}R(i)*X_r(i)+\sum_{i=1}^{i=n_w}W(i)*X_w(i)+\sum_{i=1}^{i=n_t}T(i)*X_t(i)+\sum_{i=1}^{i=n_d}D(i)*X_d(i)
\\
\text{subject to:}\qquad& \sum_{i=1}^{i=n_i} X_k(i) = 1 \\
& \sum_{i=1}^{i=n_q} X_q(i) = 2 \\
& \sum_{i=1}^{i=n_r} X_r(i) = 5 \\
& \sum_{i=1}^{i=n_w} X_w(i) = 5 \\
& \sum_{i=1}^{i=n_t} X_t(i) = 2 \\
& \sum_{i=1}^{i=n_d} X_d(i) = 1 \\
\end{aligned}
$$


## 3. Data Collected ##

The data we used in our model was found on Kaggle. It contains ESPN's 2019 fantasy football statistics and 2020 fantasy football player projections. The data is divided into position groups. We used player name and 2020 point projections for our model.

To more accurately determine each player's value, we used a metric called value over replacement player (VORP). This metric is used to measure how many points a particular player is predicted contribute to the team over a replacement-level player of the same position. For this application, we used the player that is predicted to be picked last in each position group as the replacement player. Since we are assuming that this draft is for a 10 team league, we chose the the 10th ranked kicker and defense, 20th ranked quarterback and tight end, and the 50th ranked running back and wide receiver as our replacement players. To get each player's VORP, we simply take their 2020 points prediction from ESPN and subtract the points prediction from the replacement-level player in their position group.

## 4. Solution ##

In [1]:
using CSV, DataFrames, GLPK, JuMP

In [2]:
files = readdir() # read files currently in directory 
csv_files = [] # create an empty array 

# identify all .csv files 
for i in files
    if endswith(i,".csv")
        push!(csv_files, i)
    end
end

# dictionary 
data = Dict()
names = Dict() 
key = ['D', 'K', 'Q', 'R', 'T', 'W']

index = 1
for i in csv_files
    name = key[index]
    data[name[1]] = CSV.read(i, DataFrame)[:,"2020 VORP"] # change to "2020 VORP"
    names[name[1]] = CSV.read(i, DataFrame)[:,"PLAYER NAME"]
    index += 1
end

n = Dict()
for i in key
    n[i] = length(data[i])
end

In [3]:
m = Model(with_optimizer(GLPK.Optimizer))

# define binary variables (pick player or not)
@variable(m, Xk[1:n['K']], Bin)
@variable(m, Xq[1:n['Q']], Bin)
@variable(m, Xr[1:n['R']], Bin)
@variable(m, Xw[1:n['W']], Bin)
@variable(m, Xt[1:n['T']], Bin)
@variable(m, Xd[1:n['D']], Bin)

# define points for each position 
@variable(m, k_points)
@variable(m, q_points)
@variable(m, r_points)
@variable(m, w_points)
@variable(m, t_points)
@variable(m, d_points)

# define constraints 
@constraint(m, sum(Xk) == 1)
@constraint(m, sum(Xq) == 2)
@constraint(m, sum(Xr) == 5)
@constraint(m, sum(Xw) == 5)
@constraint(m, sum(Xt) == 2)
@constraint(m, sum(Xd) == 1)

@constraint(m, k_points == sum(Xk[i]*data['K'][i] for i = 1:n['K']))
@constraint(m, q_points == sum(Xq[i]*data['Q'][i] for i = 1:n['Q']))
@constraint(m, r_points == sum(Xr[i]*data['R'][i] for i = 1:n['R']))
@constraint(m, w_points == sum(Xw[i]*data['W'][i] for i = 1:n['W']))
@constraint(m, t_points == sum(Xt[i]*data['T'][i] for i = 1:n['T']))
@constraint(m, d_points == sum(Xd[i]*data['D'][i] for i = 1:n['D']))

@objective(m, Max, k_points + q_points + r_points + w_points + t_points + d_points)
;

In [4]:
optimize!(m)

In [5]:
getobjectivevalue(m)

1925.1299999999997

In [6]:
Xk = value.(Xk)
Xq = value.(Xq) 
Xr = value.(Xr) 
Xw = value.(Xw) 
Xt = value.(Xt) 
Xd = value.(Xd) 
;

In [18]:
println("Kicker:")
for i = 1:length(Xk)
    if Xk[i] == 1
        println(names['K'][i],  ", " * string(data['K'][i]))
    end
end 
println("\nk_points = " * string(value.(k_points)))

println("\n\nQBs:")
for i = 1:length(Xq)
    if Xq[i] == 1
        println(names['Q'][i], ", " * string(data['Q'][i]))
    end
end 
println("\nq_points = " * string(value.(q_points)))

println("\n\nRBs:")
for i = 1:length(Xr)
    if Xr[i] == 1
        println(names['R'][i], ", " * string(data['R'][i]))
    end
end 
println("\nr_points = " * string(value.(r_points)))


println("\n\nWRs:")
for i = 1:length(Xw)
    if Xw[i] == 1
        println(names['W'][i], ", " * string(data['W'][i]))
    end
end 
println("\nw_points = " * string(value.(w_points)))


println("\n\nTE:")
for i = 1:length(Xt)
    if Xt[i] == 1
        println(names['T'][i], ", " * string(data['T'][i]))
    end
end 
println("\nt_points = " * string(value.(t_points)))


println("\n\nD:")
for i = 1:length(Xd)
    if Xd[i] == 1
        println(names['D'][i], ", " * string(data['D'][i]))
    end
end 
println("\nd_points = " * string(value.(d_points)))

Kicker:
Harrison Butker, 14.3

k_points = 14.300000000000004


QBs:
Lamar Jackson, 105.41
Patrick Mahomes, 99.84

q_points = 205.25


RBs:
Christian McCaffrey, 210.6
Ezekiel Elliott, 177.41
Saquon Barkley, 168.98
Dalvin Cook, 168.37
Alvin Kamara, 167.04

r_points = 892.3999999999996


WRs:
Michael Thomas, 144.99
DeAndre Hopkins, 98.85
Julio Jones, 97.5
Chris Godwin, 95.38
Davante Adams, 92.94

w_points = 529.6599999999999


TE:
Travis Kelce, 136.71
George Kittle, 117.14

t_points = 253.85000000000002


D:
Steelers D/ST, 29.67

d_points = 29.67


## 5. Results and discussion ##

### Based on our model above, we are able to pick the initial optimal fantasy football team (16 members). The results are: ###


| Kicker         | QBs            | RBs                 | WRs            | TEs          | D             |
| -------------- |:--------------:|:-------------------:|:--------------:| ------------:| -------------:|
| Harrison Butker| Lamar Jackson  | Christian McCaffrey | Michael Thomas | Travis Kelce | Steelers D/ST |
|                | Patrick Mahomes| Ezekiel Elliott     | DeAndre Hopkins| George Kittle|               |
|                |                | Saquon Barkley      | Julio Jones    |              |               |
|                |                | Dalvin Cook         | Chris Godwin   |              |               |
|                |                | Alvin Kamara        | Davante Adams  |              |               |

### From our evaluation, we see that these players are the players with highest points in our dataset. The team contains 16 players, with 1 kicker, 2 QBs, 5 RBs, 5 WRs, 2 TEs, 1 defenser. Therefore, the result matches our objective and hypothesis well. ###

We will add some more contraints and variation to build our new model in next step.